In [1]:
#importing json to read in data
import json
from pprint import pprint

import pandas as pd

In [2]:
#opening json file
with open('yelp_academic_dataset_business.json',encoding="utf8") as data_file:
    number = next(data_file).strip()
    data = [json.loads(line) for line in data_file]
    

In [3]:
#Collecting busines id for city values
#creating lists to store each city and business id
Las_vegas = []
Toronto = []
BusinessIds = []

#looping to get the records for Las Vegas and Toronto cities
for i in range(len(data)):
    if data[i]['city'] == 'Las Vegas':
        Las_vegas.append(i)
        
    if data[i]['city'] == 'Toronto':
        Toronto.append(i) 
 
#checking length is reasonable
print(len(Las_vegas))
print(len(Toronto))

#putting the businessids into one list
for i in range(len(Las_vegas)):
    BusinessIds.append(data[Las_vegas[i]]['business_id'])
    
for i in range(len(Toronto)):
    BusinessIds.append(data[Toronto[i]]['business_id'])

#ensuring no duplicated in list
Filtered_Business_Ids = list(set(BusinessIds))

print(len(Filtered_Business_Ids))



#for i in range(len(data)):
#    print(data[i]['city'])

28865
18233
47098


In [4]:
#output business ids into a json file
Business_ids = []
for i in range(len(Filtered_Business_Ids)):
    Business_ids.append(json.dumps({'Business_id': Filtered_Business_Ids[i]}))


with open('Business_ids.json', 'w') as outfile:
    json.dump(BusinessIds, outfile)

In [5]:
#checking json file
with open('Business_ids.json') as f:
    data2 = json.load(f)


In [187]:
#beginning of Fairouz's code
#move business ids to dataframe
busids = pd.read_json('Business_ids.json',lines=True,orient='index')
biz_ids = pd.DataFrame(busids)
columns = ['business_id']
biz_ids.columns = columns

In [8]:
#load reviews from feathers
import feather
reviews1 = feather.read_dataframe('reviews1.feather')
reviews2 = feather.read_dataframe('reviews2.feather')

In [188]:
#select users who commented on the selected business - remove any user duplicates
import pandas as pd
temp_user = pd.DataFrame(columns=['user_id']) 
users_list1 = pd.DataFrame(reviews1[reviews1['business_id'].isin(biz_ids['business_id'])]['user_id'])
users_list2 = pd.DataFrame(reviews2[reviews2['business_id'].isin(biz_ids['business_id'])]['user_id'])
users_list1 = users_list1.drop_duplicates(subset='user_id',keep='first')
display('list1 count: ') 
display(users_list1.count())
users_list2 = users_list2.drop_duplicates(subset='user_id',keep='first')
display('list2 count: ') 
display(users_list2.count())
users_list1 = users_list1.append(users_list2)
display('list1+2 count: ') 
display(users_list1.count())
users_list1 = users_list1.drop_duplicates(subset='user_id',keep='first')
display('list1+2 de-duplicated: ') 
display(users_list1.count())

'list1 count: '

user_id    377796
dtype: int64

'list2 count: '

user_id    299259
dtype: int64

'list1+2 count: '

user_id    677055
dtype: int64

'list1+2 de-duplicated: '

user_id    676932
dtype: int64

In [191]:
#read comments made by users accross yelp dataset
users_comments1 = pd.DataFrame(reviews1[reviews1['user_id'].isin(users_list1['user_id'])])
users_comments2 = pd.DataFrame(reviews2[reviews2['user_id'].isin(users_list1['user_id'])])
users_comments = users_comments1.append(users_comments2)
users_comments.count()

business_id    3306523
cool           3306523
date           3306523
funny          3306523
review_id      3306523
stars          3306523
text           3306523
useful         3306523
user_id        3306523
dtype: int64

In [ ]:
#use "user_comments" to run the personality analysis on users 